We split our observations into two pointings in order to cover the wavelength gap and observe more targets:
* Pointing 1: standard 3-shutter slitlets and nods
* Pointing 2: single shutter slitlets with 2 dithers + master background slitlets added by manually editing the configuration

03101: c1: 3 shutter slitlet, 1 integration each

03103: 16b...exp1: single slitlet, 2 integrations  
03105: 16b...exp2: single slitlet, 2 integrations

In [1]:
import os
home = os.path.expanduser("~")

In [2]:
import crds
crds.__version__

'11.17.22'

In [3]:
os.environ["CRDS_PATH"] = os.path.join(home, "crds_cache")
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'

In [4]:
crds.get_default_context()

ServiceError: Configured for server-less mode.  Skipping JSON RPC 'get_default_context'

In [5]:
import os
home = os.path.expanduser("~")
from glob import glob
import numpy as np
from astropy.io import fits
from astropy.table import Table, vstack, unique

In [6]:
# ---------- Set CRDS environment variables ----------
# Define CRDS paths *before* importing jwst pipeline
# otherwise you'll get FileNotFoundError: [Errno 2] :
# No such file or directory: '$HOME/crds_cache/config/jwst/server_config'

#import os
#home = os.path.expanduser("~")
import crds
#os.environ['CRDS_CONTEXT'] = 'jwst_1235.pmap'
os.environ["CRDS_PATH"] = os.path.join(home, "crds_cache")
os.makedirs(os.environ['CRDS_PATH'], exist_ok=True)
print(f'CRDS cache location: {os.environ["CRDS_PATH"]}')
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'
#os.environ['CRDS_MODE'] = 'remote'  # or 'local' if running in server-less mode
#os.environ['CRDS_MODE'] = 'local'  # server-less mode

try:
    print(f"Current Operational CRDS Context = {crds.get_default_context()}")
except Exception:
    print('CRDS server not found (are you offline?)')

CRDS cache location: /Users/dcoe/crds_cache
CRDS server not found (are you offline?)


In [7]:
print('CRDS Context:', crds.get_context_name('jwst'))

CRDS Context: jwst_1223.pmap


In [8]:
#crds.getreferences({'INSTRUMENT': 'NIRSpec'}, context='jwst_1235.pmap')

In [9]:
#from crds.list import ListScript
#ListScript().list_status()

In [10]:
# Import JWST pipeline

import jwst
from jwst import datamodels
from jwst.pipeline import Detector1Pipeline   # calwebb_detector1
from jwst.pipeline import Spec2Pipeline       # calwebb_spec2
from jwst.pipeline import Spec3Pipeline       # calwebb_spec3
from jwst.extract_1d import Extract1dStep     # Extract1D Individual Step
from jwst.associations import asn_from_list   # create association file

# Note if you're using a development version,
# it might not report the correct version number here.
print("JWST Calibration Pipeline Version = {}".format(jwst.__version__))

JWST Calibration Pipeline Version = 1.14.0


In [11]:
rate_dir = 'reprocess'

In [12]:
output_dir = 'reprocess/master-background'  # master background slitlets
os.makedirs(output_dir, exist_ok=True)

In [13]:
def allin(elements, list_or_string):
    if type(elements) == str:
        elements = elements.split()
    for element in elements:
        if element not in list_or_string:
            return False
    return True

def anyin(elements, list_or_string):
    if type(elements) == str:
        elements = elements.split()
    for element in elements:
        if element in list_or_string:
            return True
    return False

# Select subset of files containing all search strings
def select_files(all_files, search_strings=[], exclude_strings=[]):
    chosen_files = [file for file in all_files if allin(search_strings, file) and not anyin(exclude_strings, file)]
    if len(chosen_files) == 1:
        chosen_files = chosen_files[0]
    return chosen_files    

In [14]:
# Create links in output_dir directory to source files
def link_to_files(source_files, output_dir):
    links = []
    for source_file in source_files:
        link = os.path.join(output_dir, os.path.basename(source_file))
        print(link, '->', source_file)
        if not os.path.exists(link):
            os.symlink(os.path.abspath(source_file), link)
        links.append(link)
    return links

In [15]:
def is_negative(x):
    return x < 0

def filter_table(full_table, **kwargs):
    """
    Filters an Astropy Table based on the given pairs of column,value or column,function.

    Parameters:
    - full_table (Table): The input Astropy Table.
    - **kwargs: Arbitrary number of pairs for filtering. 
                Each should be in the format column_name=value (or function)

    Returns:
    - Table: The filtered Astropy Table.

    Example Usage:
    >>> filtered_table = filter_table(full_table, column1=value1, column2=function2)
    """
    filtered_table = full_table  # start with the full table
    for column, condition in kwargs.items():
        if callable(condition):  # Check if the condition is a function
            filtered_table = filtered_table[condition(filtered_table[column])]
        else:
            filtered_table = filtered_table[filtered_table[column] == condition]        
    return filtered_table

In [16]:
rate_files = sorted(glob(os.path.join(rate_dir,    '*_rate.fits')))
rate_files = select_files(rate_files, ['_03105'])  # 03101 03103 03105
rate_files

['reprocess/jw04246003001_03105_00001_nrs1_rate.fits',
 'reprocess/jw04246003001_03105_00001_nrs2_rate.fits',
 'reprocess/test_jw04246003001_03105_00001_nrs2_rate.fits']

In [17]:
link_to_files(rate_files, output_dir)

reprocess/master-background/jw04246003001_03105_00001_nrs1_rate.fits -> reprocess/jw04246003001_03105_00001_nrs1_rate.fits
reprocess/master-background/jw04246003001_03105_00001_nrs2_rate.fits -> reprocess/jw04246003001_03105_00001_nrs2_rate.fits
reprocess/master-background/test_jw04246003001_03105_00001_nrs2_rate.fits -> reprocess/test_jw04246003001_03105_00001_nrs2_rate.fits


['reprocess/master-background/jw04246003001_03105_00001_nrs1_rate.fits',
 'reprocess/master-background/jw04246003001_03105_00001_nrs2_rate.fits',
 'reprocess/master-background/test_jw04246003001_03105_00001_nrs2_rate.fits']

In [18]:
rate_file_links = link_to_files(rate_files, output_dir)
rate_file_links

reprocess/master-background/jw04246003001_03105_00001_nrs1_rate.fits -> reprocess/jw04246003001_03105_00001_nrs1_rate.fits
reprocess/master-background/jw04246003001_03105_00001_nrs2_rate.fits -> reprocess/jw04246003001_03105_00001_nrs2_rate.fits
reprocess/master-background/test_jw04246003001_03105_00001_nrs2_rate.fits -> reprocess/test_jw04246003001_03105_00001_nrs2_rate.fits


['reprocess/master-background/jw04246003001_03105_00001_nrs1_rate.fits',
 'reprocess/master-background/jw04246003001_03105_00001_nrs2_rate.fits',
 'reprocess/master-background/test_jw04246003001_03105_00001_nrs2_rate.fits']

In [19]:
rate_files

['reprocess/jw04246003001_03105_00001_nrs1_rate.fits',
 'reprocess/jw04246003001_03105_00001_nrs2_rate.fits',
 'reprocess/test_jw04246003001_03105_00001_nrs2_rate.fits']

In [20]:
msa_metafiles = []
for rate_file in rate_files:
    try:
        msa_metafile = fits.getval(rate_file, 'MSAMETFL')
        print(os.path.basename(rate_file), msa_metafile)
        msa_metafiles.append(msa_metafile)
    except:
        print(os.path.basename(rate_file))

jw04246003001_03105_00001_nrs1_rate.fits jw04246003001_03_msa.fits
jw04246003001_03105_00001_nrs2_rate.fits jw04246003001_03_msa.fits
test_jw04246003001_03105_00001_nrs2_rate.fits jw04246003001_03_msa.fits


In [21]:
#msa_metafiles = sorted(glob(os.path.join(rate_dir,  '*_msa.fits')))
#msa_metafiles

In [22]:
#for msa_metafile in msa_metafiles:
#    msa_hdu_list = fits.open(msa_metafile)
#    msa_hdu_list.info()

In [23]:
msa_hdu_list = fits.open(os.path.join(rate_dir, msa_metafile))
msa_hdu_list.info()

Filename: reprocess/jw04246003001_03_msa.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      16   ()      
  1  SHUTTER_IMAGE    1 ImageHDU         8   (342, 730)   int16   
  2  SHUTTER_INFO    1 BinTableHDU     33   636R x 12C   [I, I, I, I, I, J, 1A, 6A, E, E, I, 1A]   
  3  SOURCE_INFO    1 BinTableHDU     25   179R x 8C   [J, J, 20A, 31A, D, D, 30A, D]   


In [24]:
shutter_table = Table(msa_hdu_list['SHUTTER_INFO'].data)
source_table  = Table(msa_hdu_list['SOURCE_INFO'].data)

In [25]:
set(shutter_table['msa_metadata_id'])

{1, 46, 146}

In [26]:
#master_background_shutter_table = shutter_table_dither1[shutter_table_dither1['source_id'] < 0]

master_background_shutter_table = filter_table(shutter_table, 
                                               #dither_point_index = 1, # There is only one
                                               msa_metadata_id = 146, 
                                               source_id = lambda x: x < 0)  # is_negative

# for longer multiple slitlets, pipeline tries to define primary_source in the middle (or close to it) in dither 1
# ignore all that, and define them all as primary_source, not background
master_background_shutter_table['primary_source'] = 'Y'
master_background_shutter_table['background'] = 'N'
master_background_shutter_table[:3]

slitlet_id,msa_metadata_id,shutter_quadrant,shutter_row,shutter_column,source_id,background,shutter_state,estimated_source_in_shutter_x,estimated_source_in_shutter_y,dither_point_index,primary_source
int16,int16,int16,int16,int16,int32,str1,str6,float32,float32,int16,str1
162,146,2,341,144,-2,N,OPEN,0.5,0.5,1,Y
164,146,2,343,104,-3,N,OPEN,0.5,0.5,1,Y
213,146,3,250,62,-4,N,OPEN,0.5,0.5,1,Y


In [27]:
master_background_source_table = filter_table(source_table, source_id = lambda x: x < 0)
master_background_source_table[:3]

program,source_id,source_name,alias,ra,dec,preimage_id,stellarity
int32,int32,str20,str31,float64,float64,str30,float64
4246,-38,4246_-38,06:48:0.3377 +70:15:1.80,102.0014072688316,70.25050078700221,04246001_001,0.0
4246,-37,4246_-37,06:48:0.3925 +70:14:54.22,102.0016355278552,70.24839575221984,04246001_001,0.0
4246,-36,4246_-36,06:47:59.3266 +70:14:56.62,101.9971942814473,70.24906102776545,04246001_001,0.0


In [28]:
output_dir

'reprocess/master-background'

In [29]:
msa_metafile

'jw04246003001_03_msa.fits'

In [30]:
msa_hdu_list.info()

Filename: reprocess/jw04246003001_03_msa.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      16   ()      
  1  SHUTTER_IMAGE    1 ImageHDU         8   (342, 730)   int16   
  2  SHUTTER_INFO    1 BinTableHDU     33   636R x 12C   ['I', 'I', 'I', 'I', 'I', 'J', '1A', '6A', 'E', 'E', 'I', '1A']   
  3  SOURCE_INFO    1 BinTableHDU     25   179R x 8C   ['J', 'J', '20A', '31A', 'D', 'D', '30A', 'D']   


In [31]:
msa_hdu_list['SHUTTER_INFO'] = fits.table_to_hdu(master_background_shutter_table)
msa_hdu_list['SOURCE_INFO']  = fits.table_to_hdu(master_background_source_table)

msa_hdu_list[2].name = 'SHUTTER_INFO'
msa_hdu_list[3].name = 'SOURCE_INFO'

msa_hdu_list.info()

Filename: reprocess/jw04246003001_03_msa.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      16   ()      
  1  SHUTTER_IMAGE    1 ImageHDU         8   (342, 730)   int16   
  2  SHUTTER_INFO    1 BinTableHDU     33   47R x 12C   ['I', 'I', 'I', 'I', 'I', 'J', '1A', '6A', 'E', 'E', 'I', '1A']   
  3  SOURCE_INFO    1 BinTableHDU     25   38R x 8C   ['J', 'J', '20A', '31A', 'D', 'D', '30A', 'D']   


In [32]:
output_msa_metafile = os.path.join(output_dir, msa_metafile)
print(output_msa_metafile)
msa_hdu_list.writeto(output_msa_metafile, overwrite=True)

reprocess/master-background/jw04246003001_03_msa.fits


# Spec2

In [33]:
rate_file_links

['reprocess/master-background/jw04246003001_03105_00001_nrs1_rate.fits',
 'reprocess/master-background/jw04246003001_03105_00001_nrs2_rate.fits',
 'reprocess/master-background/test_jw04246003001_03105_00001_nrs2_rate.fits']

In [34]:
params_spec2 = {}
params_spec2['resample_spec'] = {'weight_type':'ivm'} # pixel-based inverse read noise (vs. 'exptime' single value exposure time)
params_spec2['master_background_mos'] = {'skip':True} # Need to set explicitly one way or the other. Otherwise, default skips exposure 1 but runs on exposures 2 and 3.
params_spec2

{'resample_spec': {'weight_type': 'ivm'},
 'master_background_mos': {'skip': True}}

In [35]:
for rate_file_link in rate_file_links:
    print(rate_file_link)
    spec2_result = Spec2Pipeline.call(
            rate_file_link,
            steps=params_spec2,
            output_dir=output_dir,
            save_results=True,
        )

reprocess/master-background/jw04246003001_03105_00001_nrs1_rate.fits


2024-05-30 18:39:48,336 - CRDS - ERROR -  Error determining best reference for 'pars-nscleanstep'  =   Unknown reference type 'pars-nscleanstep'
2024-05-30 18:39:48,373 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.
2024-05-30 18:39:48,374 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2024-05-30 18:39:48,375 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.
2024-05-30 18:39:48,376 - stpipe.Spec2Pipeline.nsclean - INFO - NSCleanStep instance created.
2024-05-30 18:39:48,377 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2024-05-30 18:39:48,378 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.
2024-05-30 18:39:48,379 - stpipe.Spec2Pipeline.extract_2d - INFO - Extract2dStep instance created.
2024-05-30 18:39:48,381 - stpipe.Spec2Pipeline.master_background_mos - INFO - MasterBackgroundMosStep instance created.
2024-05-30 18:39:48,382 - stpipe.Spec2Pipeline.

2024-05-30 18:39:48,547 - stpipe.Spec2Pipeline - INFO - Prefetching reference files for dataset: 'jw04246003001_03105_00001_nrs1_rate.fits' reftypes = ['apcorr', 'area', 'barshadow', 'camera', 'collimator', 'cubepar', 'dflat', 'disperser', 'distortion', 'drizpars', 'extract1d', 'fflat', 'filteroffset', 'flat', 'fore', 'fpa', 'fringe', 'ifufore', 'ifupost', 'ifuslicer', 'mrsxartcorr', 'msa', 'msaoper', 'ote', 'pathloss', 'photom', 'regions', 'sflat', 'speckernel', 'specprofile', 'spectrace', 'specwcs', 'wavecorr', 'wavelengthrange', 'wavemap', 'wfssbkg']


CrdsNetworkError: Failed downloading cache config from: JSON RPC service at 'https://crds-serverless-mode.stsci.edu': "Configured for server-less mode.  Skipping JSON RPC 'get_server_info'"

In [39]:
rate_file_link

'reprocess/master-background/jw04246003001_03105_00001_nrs1_rate.fits'

In [37]:
test_rate_file = 'reprocess/test_jw04246003001_03105_00001_nrs2_rate.fits'

In [40]:
if 0:
    spec2_result = Spec2Pipeline.call(
            test_rate_file,
            steps=params_spec2,
            output_dir=output_dir,
            save_results=True,
        )

In [36]:
if 0: #for rate_file_link in rate_file_links:
    try:
        spec2pipe = Spec2Pipeline()
        #spec2pipe.resample_spec.weight_type = 'exptime' # better for high SNR data – see NS-MOS07 https://jwst-docs.stsci.edu/jwst-calibration-pipeline-caveats/jwst-nirspec-mos-pipeline-caveats
        #spec2pipe.pathloss.skip = True  # Skip path loss corrections
        #spec2pipe.nsclean.skip = False  # Run NSClean 1/f noise striping removal (implemented in pipeline version 1.12.6 but off by default)
        spec2pipe.master_background_mos.skip = True  # Need to set explicitly one way or the other. Otherwise, default skips exposure 1 but runs on exposures 2 and 3.
        spec2pipe.save_results = True
        spec2pipe.output_dir = subset_dir
        spec2_result = spec2pipe(rate_file_link)
    except:
        print('spec2pipe FAILED on', rate_file_link)    